<a href="https://colab.research.google.com/github/danjshaw/ece57000-finalProject/blob/main/lora-tiny-bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Setup

In [1]:
!pip install datasets
!pip install evaluate
!pip freeze > requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import numpy as np
from transformers import set_seed
set_seed(0)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


Followed examples from this [Hugging Face NLP course](https://huggingface.co/learn/nlp-course/chapter3/3?fw=pt#fine-tuning-a-model-with-the-trainer-api) on how to use the trainer API for fine-tuning.

In [8]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [9]:
import evaluate

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
num_runs = 5

Hyperparameters from [google-research/bert](https://github.com/google-research/bert):



> For each task, we selected the best fine-tuning hyperparameters from the lists below, and trained for 4 epochs:
> * batch sizes: 8, 16, 32, 64, 128
> * learning rates: 3e-4, 1e-4, 5e-5, 3e-5



In [ ]:
epochs = 4
batch_sizes = [8, 16, 32, 64, 128]
learning_rates = [3e-4, 1e-4, 5e-5, 3e-5]

# Full Fine-Tuning



In [ ]:
from transformers import AutoModelForSequenceClassification

ft_model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

In [ ]:
from transformers import Trainer, TrainingArguments

ft_results = []
iterations = len(batch_sizes)*len(learning_rates)*num_runs
for run in range(1, num_runs+1):
  progress = 0
  for size in batch_sizes:
    for rate in learning_rates:
      progress += 1
      print(f"Progress: {progress}/{iterations}")
      ft_result = {"run":run, "batch_size": size, "learning_rate": rate}

      training_args = TrainingArguments(
          "ft-trainer",
          eval_strategy="epoch",
          per_device_eval_batch_size=size,
          per_device_train_batch_size=size,
          num_train_epochs=epochs,
          learning_rate=rate,
          disable_tqdm=True,
          report_to="none"
      )

      trainer = Trainer(
          ft_model,
          training_args,
          train_dataset=tokenized_datasets["train"],
          eval_dataset=tokenized_datasets["validation"],
          data_collator=data_collator,
          processing_class=tokenizer,
          compute_metrics=compute_metrics
      )

      ft_result = ft_result | trainer.train().metrics | trainer.evaluate()
      ft_results.append(ft_result)

ft_max = ft_results[0]
for i, result in enumerate(ft_results):
  if result['eval_f1'] > ft_max['eval_f1']:
    ft_max = result

print(f'\n\n================\
        \nBest Result: \
        \nbatch_size={ft_max["batch_size"]}, \
        \nlearning_rate={ft_max["learning_rate"]}, \
        \neval_f1={ft_max["eval_f1"]}')

In [ ]:
import csv

with open('ft-results.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=list(ft_results[0].keys()))

    writer.writeheader()
    for result in ft_results:
      writer.writerow(result)

# Low-Rank Adaptation (LoRA)

In [4]:
class LoraModule(nn.Module):
  def __init__(self, in_features, out_features, rank, alpha):
    super().__init__()
    self.scale = alpha / rank
    self.A = nn.Parameter(torch.randn(in_features, rank))
    self.B = nn.Parameter(torch.zeros(rank, out_features))

  def forward(self, x):
    return (self.scale * (x @ self.A @ self.B))

class LoraLinear(nn.Module):
  def __init__(self, linear, rank, alpha):
    super().__init__()
    if (isinstance(linear, LoraLinear)):
      self.linear = linear.linear
      self.lora = LoraModule(self.linear.in_features, self.linear.out_features, rank, alpha)

    else:
      self.linear = linear
      self.lora = LoraModule(self.linear.in_features, self.linear.out_features, rank, alpha)

  def forward(self, x):
    return self.linear(x) + self.lora(x)

In [5]:
def get_trainable_parameters(model):
  trainable_parameters = 0
  parameters = 0
  for param in model.parameters():
    parameters += param.numel()
    if param.requires_grad:
      trainable_parameters += param.numel()
  return parameters, trainable_parameters

In [6]:
def configure_lora_model(model, rank, alpha):
  for i, layer in enumerate(model.bert.encoder.layer):
    s = layer.attention.self
    s.query = LoraLinear(s.query, rank, alpha)
    s.value = LoraLinear(s.value, rank, alpha)

  for name, param in model.named_parameters():
    if 'A' in name or 'B' in name:
      param.requires_grad = True
    else:
      param.requires_grad = False
  params, trainable_params = get_trainable_parameters(model)
  print(f"Total Parameters={params}, Trainable Parameters={trainable_params}")

  return (params, trainable_params)

In [7]:
from transformers import AutoModelForSequenceClassification

lora_model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

NameError: name 'checkpoint' is not defined

In [ ]:
ranks = [1, 2, 4, 8, 16]
alphas = [1, 2, 4, 8, 16]

In [ ]:
from transformers import Trainer, TrainingArguments

results = []
iterations = len(batch_sizes)*len(learning_rates)*len(ranks)*len(alphas)*num_runs
progress = 0
for run in range(1, num_runs+1):
  for rank in ranks:
    for alpha in alphas:
      for size in batch_sizes:
        for rate in learning_rates:
          progress += 1
          print(f"Progress: {progress}/{iterations}")
          result = {"run": run, "rank": rank, "alpha": alpha, "batch_size": size, "learning_rate": rate}

          params, trainable_params = configure_lora_model(lora_model, rank, alpha)
          result["parameters"] = params
          result["trainable_parameters"] = trainable_params

          training_args = TrainingArguments(
              "lora-trainer",
              eval_strategy="epoch",
              per_device_eval_batch_size=size,
              per_device_train_batch_size=size,
              num_train_epochs=epochs,
              learning_rate=rate,
              disable_tqdm=True,
              report_to="none"
          )

          trainer = Trainer(
              lora_model,
              training_args,
              train_dataset=tokenized_datasets["train"],
              eval_dataset=tokenized_datasets["validation"],
              data_collator=data_collator,
              processing_class=tokenizer,
              compute_metrics=compute_metrics
          )

          result = result | trainer.train().metrics | trainer.evaluate()
          results.append(result)

max = results[0]
for i, result in enumerate(results):
  if result['eval_f1'] > max['eval_f1']:
    max = result

print(f'\n\n================\
        \nBest Result: \
        \nrank={max["rank"]}, \
        \nalpha={max["alpha"]}, \
        \nparameters={max["parameters"]}, \
        \ntrainable_parameters={max["trainable_parameters"]}, \
        \nbatch_size={max["batch_size"]}, \
        \nlearning_rate={max["learning_rate"]}, \
        \neval_f1={max["eval_f1"]}')

In [ ]:
import csv

with open('lora-results.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=list(results[0].keys()))

    writer.writeheader()
    for result in results:
      writer.writerow(result)